In [1]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
def closure_graph(synset,fn):
    seen = set()
    node = set()
    edge = list()
    def recurse(s):
        if not s in seen:
            seen.add(s)
            node.add(s.lemma_names('eng')[0])
            for i in range(1, len(s.lemma_names('eng'))):
                node.add(s.lemma_names('eng')[i])
                edge.append({"source":s.lemma_names('eng')[0],"target":s.lemma_names('eng')[i],"type":"synonym"})
            for s1 in fn(s):
                node.add(s1.lemma_names('eng')[0])
                edge.append({"source":s.lemma_names('eng')[0],"target":s1.lemma_names('eng')[0],"type":"hypernym"})
                recurse(s1)
    recurse(synset)
    return (list(node), edge)

In [3]:
import json

dog = wn.synset('dog.n.01')
graph = closure_graph(dog,lambda s: s.hypernyms())
print(graph)
nodes = []
nodesJson = {}
for i in range(len(graph[0])):
    nodes.append({"id":i+1, "name": graph[0][i]})
    nodesJson[graph[0][i]] = i+1
links = []
for i in range(len(graph[1])):
    graph[1][i]['source'] = nodesJson[graph[1][i]['source']]
    graph[1][i]['target'] = nodesJson[graph[1][i]['target']]
    links.append(graph[1][i])
with open("wordnet.json","w") as fh:
    json.dump({"nodes":nodes,"links":graph[1]},fh)

(['animal', 'eutherian_mammal', 'entity', 'whole', 'brute', 'vertebrate', 'animate_thing', 'placental_mammal', 'domesticated_animal', 'canid', 'domestic_dog', 'fauna', 'object', 'chordate', 'physical_object', 'unit', 'canine', 'being', 'craniate', 'beast', 'domestic_animal', 'carnivore', 'physical_entity', 'creature', 'placental', 'eutherian', 'animate_being', 'mammalian', 'Canis_familiaris', 'mammal', 'living_thing', 'dog', 'organism'], [{'source': 'dog', 'target': 'domestic_dog', 'type': 'synonym'}, {'source': 'dog', 'target': 'Canis_familiaris', 'type': 'synonym'}, {'source': 'dog', 'target': 'canine', 'type': 'hypernym'}, {'source': 'canine', 'target': 'canid', 'type': 'synonym'}, {'source': 'canine', 'target': 'carnivore', 'type': 'hypernym'}, {'source': 'carnivore', 'target': 'placental', 'type': 'hypernym'}, {'source': 'placental', 'target': 'placental_mammal', 'type': 'synonym'}, {'source': 'placental', 'target': 'eutherian', 'type': 'synonym'}, {'source': 'placental', 'target'